In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
sns.set(font_scale=1.4)
sns.set_style("whitegrid")

In [2]:
playlist_df = pd.read_csv('./playlists/favourites.csv', 
                          names=['track', 'artist', 'album', 'playlist_name', 'type'], header=0)

playlist_df.head(20)

,track,artist,album,playlist_name,type
0,The Masterplan (Remastered),Oasis,(What's The Story) Morning Glory? [Remastered],Favourites 2019/02/20,Playlist track
1,Don't Look Back in Anger (Remastered),Oasis,(What's The Story) Morning Glory? [Remastered],Favourites 2019/02/20,Playlist track
2,Breezeblocks,alt-J,An Awesome Wave,Favourites 2019/02/20,Playlist track
3,Golden Skans,Klaxons,Golden Skans,Favourites 2019/02/20,Playlist track
4,I Wanna Be Yours,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
5,R U Mine?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
6,Do I Wanna Know?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
7,Fluorescent Adolescent,Arctic Monkeys,Favourite Worst Nightmare,Favourites 2019/02/20,Playlist track
8,Snap Out of It,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
9,Cast No Shadow (Remastered),Oasis,(What's The Story) Morning Glory? [Remastered],Favourites 2019/02/20,Playlist track


In [3]:
playlist_df['track_lower'] = playlist_df['track'].str.lower()
playlist_df.drop_duplicates(subset='track_lower', keep='first', inplace=True)
playlist_df.drop('track_lower', axis=1, inplace=True)

In [4]:
artist_counts = playlist_df['artist'].value_counts()
artist_counts

Arctic Monkeys                        24
Oasis                                 18
Gorillaz                              17
Noel Gallagher's High Flying Birds    14
Twenty One Pilots                     11
Paramore                              10
Panic! At the Disco                    9
The Wombats                            8
The Neighbourhood                      6
The Killers                            6
Sigrid                                 6
Red Hot Chili Peppers                  6
The Last Shadow Puppets                5
MGMT                                   5
Jake Bugg                              5
Liam Gallagher                         4
Walk The Moon                          4
Conan Gray                             4
The Strokes                            4
The Beatles                            4
Echosmith                              3
Capital Cities                         3
Imagine Dragons                        3
Fall Out Boy                           3
Tame Impala     

In [5]:
len([x for x in artist_counts if x == 1]), len(artist_counts)

(114, 163)

In [6]:
album_counts = playlist_df['album'].value_counts()
album_counts.head(10)

AM                                                9
Definitely Maybe (Remastered)                     8
The Now Now                                       6
Blurryface                                        5
Trench                                            5
Tranquility Base Hotel & Casino                   5
After Laughter                                    4
(What's The Story) Morning Glory? [Remastered]    4
As You Were (Deluxe Edition)                      4
Sunset Season                                     4
Name: album, dtype: int64

In [7]:
playlist_df.loc[playlist_df['album'] == 'AM']

,track,artist,album,playlist_name,type
4,I Wanna Be Yours,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
5,R U Mine?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
6,Do I Wanna Know?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
8,Snap Out of It,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
14,One for the Road,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
25,Fireside,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
103,Why'd You Only Call Me When You're High?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
109,Knee Socks,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track
150,No.1 Party Anthem,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track


In [8]:
import json
import urllib


API_KEY = '4e9769f8472b284b5273ac260aedc222'

def trackInfoUrl(artist, track):
    return "http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key=%s&artist=%s&track=%s&format=json"%(API_KEY, artist, track)

In [9]:
with urllib.request.urlopen(trackInfoUrl('Arctic Monkeys', 'Batphone')) as url:
    data = json.loads(url.read().decode())
data

{'track': {'name': 'Batphone',
  'url': 'https://www.last.fm/music/Arctic+Monkeys/_/Batphone',
  'duration': '271000',
  'streamable': {'#text': '0', 'fulltrack': '0'},
  'listeners': '60196',
  'playcount': '393605',
  'artist': {'name': 'Arctic Monkeys',
   'mbid': 'ada7a83c-e3e1-40f1-93f9-3e73dbc9298a',
   'url': 'https://www.last.fm/music/Arctic+Monkeys'},
  'toptags': {'tag': [{'name': 'rock', 'url': 'https://www.last.fm/tag/rock'},
    {'name': 'british', 'url': 'https://www.last.fm/tag/british'},
    {'name': 'alternative', 'url': 'https://www.last.fm/tag/alternative'},
    {'name': 'alternative rock',
     'url': 'https://www.last.fm/tag/alternative+rock'},
    {'name': 'pop rock', 'url': 'https://www.last.fm/tag/pop+rock'}]}}}

In [10]:
271000 / 60 / 1000

4.516666666666667

In [11]:
tagged_df = playlist_df.copy()
tagged_df['duration'] = tagged_df['playcount_lastfm'] = 0

In [12]:
track, artist, album = tagged_df.loc[[0]].values.T.reshape(-1)[:3]
track, artist, album

('The Masterplan (Remastered)',
 'Oasis',
 "(What's The Story) Morning Glory? [Remastered]")

In [13]:
for row in playlist_df.itertuples():
    i = row.Index
    track, artist, album = tagged_df.loc[[i]].values.T.reshape(-1)[:3]
    track = urllib.parse.quote(track, safe='')
    artist = urllib.parse.quote(artist, safe='')
    try:
        with urllib.request.urlopen(trackInfoUrl(artist, track)) as url:
            track_info = json.loads(url.read().decode())['track']
            tagged_df.loc[[i], 'duration'] = track_info['duration']
            tagged_df.loc[[i], 'playcount_lastfm'] = track_info['playcount']
            tags = track_info['toptags']['tag']
            for tag in tags:
                tag_name = tag['name'] 
                if tag_name not in tagged_df.columns:
                    tagged_df[tag_name] = 0
                tagged_df.loc[[i], tag_name] = 1
    except:
        print('failed', end=' ')
    print(i, end=' ')
    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 failed 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 failed 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 252 255 256 257 258 259 260 261 262 263 264 265 266 267 268 271 272 273 274 275 277 278 279 280 281 282 

In [14]:
tagged_df.head(20)

,track,artist,album,playlist_name,type,duration,playcount_lastfm,played,indie,alternative,...,Crap,Perfection,new 2015,brazilian rapper,beyoncé,avião,rap brasileiro,dark,indy,vibey
0,The Masterplan (Remastered),Oasis,(What's The Story) Morning Glory? [Remastered],Favourites 2019/02/20,Playlist track,325000,4189,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Don't Look Back in Anger (Remastered),Oasis,(What's The Story) Morning Glory? [Remastered],Favourites 2019/02/20,Playlist track,0,49450,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Breezeblocks,alt-J,An Awesome Wave,Favourites 2019/02/20,Playlist track,227000,7648204,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Golden Skans,Klaxons,Golden Skans,Favourites 2019/02/20,Playlist track,165000,5293117,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,I Wanna Be Yours,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track,184000,4370394,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,R U Mine?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track,202000,9573281,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,Do I Wanna Know?,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track,272000,12219410,0,1,1,...,0,0,0,0,0,0,0,0,0,0
7,Fluorescent Adolescent,Arctic Monkeys,Favourite Worst Nightmare,Favourites 2019/02/20,Playlist track,177000,10274688,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8,Snap Out of It,Arctic Monkeys,AM,Favourites 2019/02/20,Playlist track,193000,4137071,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,Cast No Shadow (Remastered),Oasis,(What's The Story) Morning Glory? [Remastered],Favourites 2019/02/20,Playlist track,0,12876,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
top_tags = tagged_df.iloc[:,8:].apply(pd.value_counts).loc[[1]].sort_values(by=[1], axis=1, ascending=False)
top_tags

,rock,alternative,indie,british,pop,indie rock,alternative rock,electronic,indie pop,american,...,summer 2014,Am I Wrong,phillip phillips,wea,tropical house,gospel choir,amazing,blues rock,hardcore,vibey
1,115,113,102,64,64,62,52,38,27,20,...,1,1,1,1,1,1,1,1,1,1


In [28]:
tagged_df.to_csv('./playlists/tagged_favourites.csv', index=None)

In [40]:
for c in top_tags:
    print(c, ' ', top_tags.loc[[1], [c]].values[0,0])

rock   115
alternative   113
indie   102
british   64
pop   64
indie rock   62
alternative rock   52
electronic   38
indie pop   27
american   20
pop rock   19
love at first listen   16
new wave   15
Hip-Hop   15
britpop   14
rap   13
electropop   12
dance   10
2018 single   9
folk   8
gorillaz   7
90s   7
funk   7
80s   7
psychedelic   7
Love   7
synthpop   7
classic rock   6
Red Hot Chili Peppers   6
2017 single   6
2017   6
alternative pop   5
2010s   5
10s   5
hip hop   5
FM4   5
oasis   5
female vocalists   5
pop punk   5
fallon   4
Lo-Fi   4
2014   4
2013   4
Disco   4
post-punk   4
australian   4
punk rock   4
USA   3
The Killers   3
jazz   3
2016 single   3
jake bugg   3
chill   3
Canadian   3
alternative hip hop   3
summer   3
blues   3
indietronica   3
arctic monkeys   3
catchy   3
male vocalists   3
beautiful   3
punk   3
acoustic   3
hard rock   3
cover   3
fip   3
commented   3
rnb   3
duo   3
soul   3
2015 single   3
bbc radio1 playlist 2018   2
Queen   2
sex   2
psychede